In [199]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import LabelEncoder
from scipy.stats import randint
from sklearn.base import clone
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold
from numpy import ravel

In [3]:
train_set=pd.read_csv('train.csv')
test_set=pd.read_csv('test.csv')

In [8]:
lb=LabelEncoder()

In [9]:
train_set.isnull().sum()

id                              0
Time_spent_Alone             1190
Stage_fear                   1893
Social_event_attendance      1180
Going_outside                1466
Drained_after_socializing    1149
Friends_circle_size          1054
Post_frequency               1264
Personality                     0
dtype: int64

In [15]:
train_set['id'].nunique()==train_set.shape[0]

True

In [22]:
train_set.columns

Index(['id', 'Time_spent_Alone', 'Stage_fear', 'Social_event_attendance',
       'Going_outside', 'Drained_after_socializing', 'Friends_circle_size',
       'Post_frequency', 'Personality'],
      dtype='object')

In [23]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18524 entries, 0 to 18523
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         18524 non-null  int64  
 1   Time_spent_Alone           17334 non-null  float64
 2   Stage_fear                 16631 non-null  object 
 3   Social_event_attendance    17344 non-null  float64
 4   Going_outside              17058 non-null  float64
 5   Drained_after_socializing  17375 non-null  object 
 6   Friends_circle_size        17470 non-null  float64
 7   Post_frequency             17260 non-null  float64
 8   Personality                18524 non-null  object 
dtypes: float64(5), int64(1), object(3)
memory usage: 1.3+ MB


In [68]:
# 空值处理

In [69]:
numeric_cols=['Time_spent_Alone','Social_event_attendance',
       'Going_outside', 'Friends_circle_size','Post_frequency']
unnumeric_cols=['Stage_fear','Drained_after_socializing']

In [81]:
def process_numeric_cols(df):
       for col in numeric_cols:
              # 新建一个标识该列为空的列
              col_add=col+'_isnull'
              df[col_add]=0
              df.loc[df[col].isnull(),col_add]=1
              # 补空值
              df[col]=df[col].fillna(df[col].quantile(q=0.5))
       return df
def process_unnumeric_cols(df):
       for col in unnumeric_cols:
              df[col]=lb.fit_transform(df[col])
       return df
def process_target_cols(df):
       df['Personality']=lb.fit_transform(df['Personality'])
       return df

In [181]:
train_pre1=process_numeric_cols(train_set.copy())
train_pre2=process_unnumeric_cols(train_pre1.copy())
train_data=process_target_cols(train_pre2.copy())

test_pre1=process_numeric_cols(test_set.copy())
test_data=process_unnumeric_cols(test_pre1.copy())

In [87]:
feature_col=['Time_spent_Alone', 'Stage_fear', 'Social_event_attendance',
       'Going_outside', 'Drained_after_socializing', 'Friends_circle_size',
       'Post_frequency', 'Time_spent_Alone_isnull',
       'Social_event_attendance_isnull', 'Going_outside_isnull',
       'Friends_circle_size_isnull', 'Post_frequency_isnull']
target_col=['Personality']

In [182]:
# 训练数据
X_train=train_data[feature_col].values
y_train=train_data[target_col].values
# 测试数据
X_test=test_data[feature_col].values

In [153]:
# 1、随机森林
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [96]:
rf = RandomForestClassifier()

In [98]:
param_grid = {'n_estimators': [50, 100,150, 200],
              'max_depth': randint(5,10),
              'min_samples_split': randint(2,30),
              'min_samples_leaf': randint(1,30),
              'max_features':[0.1,0.3,0.5,0.7,0.9]
              }

In [149]:
rsearch.cv_results_['params'][5]

{'max_depth': 5,
 'max_features': 0.1,
 'min_samples_leaf': 24,
 'min_samples_split': 27,
 'n_estimators': 100}

In [151]:
rsearch.cv_results_['params']

[{'max_depth': 6,
  'max_features': 0.5,
  'min_samples_leaf': 23,
  'min_samples_split': 14,
  'n_estimators': 150},
 {'max_depth': 9,
  'max_features': 0.1,
  'min_samples_leaf': 9,
  'min_samples_split': 3,
  'n_estimators': 200},
 {'max_depth': 9,
  'max_features': 0.3,
  'min_samples_leaf': 11,
  'min_samples_split': 11,
  'n_estimators': 200},
 {'max_depth': 5,
  'max_features': 0.5,
  'min_samples_leaf': 21,
  'min_samples_split': 29,
  'n_estimators': 200},
 {'max_depth': 7,
  'max_features': 0.7,
  'min_samples_leaf': 19,
  'min_samples_split': 9,
  'n_estimators': 200},
 {'max_depth': 5,
  'max_features': 0.1,
  'min_samples_leaf': 24,
  'min_samples_split': 27,
  'n_estimators': 100},
 {'max_depth': 8,
  'max_features': 0.5,
  'min_samples_leaf': 21,
  'min_samples_split': 17,
  'n_estimators': 50},
 {'max_depth': 5,
  'max_features': 0.9,
  'min_samples_leaf': 15,
  'min_samples_split': 7,
  'n_estimators': 100},
 {'max_depth': 6,
  'max_features': 0.1,
  'min_samples_leaf'

In [152]:
best_estimator

RandomForestClassifier(max_depth=5, max_features=0.9, min_samples_leaf=15,
                       min_samples_split=7)

In [165]:
best_estimator=rsearch.best_estimator_

In [225]:
def result_tocsv(arr):
    result=test_data[['id']]
    result['Personality']=arr
    map_dict={0:'Extrovert',1:'Introvert'}
    result['Personality']=result['Personality'].map(map_dict)
    result.to_csv('result.csv',index=False)
    print('结果写出')
    


In [156]:
param_grid = {'n_estimators': [50, 100,150, 200],
              'max_depth': [5,10,15,20],
              'min_samples_split': [2,5,10,15,20],
              'min_samples_leaf': [1,3,5,7,9],
              'max_features':[0.1,0.3,0.5,0.7,0.9]
              }

In [157]:
gsearch=GridSearchCV(estimator=rf,param_grid=param_grid,scoring='roc_auc',cv=5)

In [158]:
gsearch.fit(X_train,y_train)

D:\tech_softwar\miniconda\envs\for_autogluon\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
D:\tech_softwar\miniconda\envs\for_autogluon\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
D:\tech_softwar\miniconda\envs\for_autogluon\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
D:\tech_softwar\miniconda\envs\for_autogluon\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [5, 10, 15, 20],
                         'max_features': [0.1, 0.3, 0.5, 0.7, 0.9],
                         'min_samples_leaf': [1, 3, 5, 7, 9],
                         'min_samples_split': [2, 5, 10, 15, 20],
                         'n_estimators': [50, 100, 150, 200]},
             scoring='roc_auc')

In [159]:
'max_depth': 5,
 'max_features': 0.5,
 'min_samples_leaf': 5,
 'min_samples_split': 5,
 'n_estimators': 100

{'max_depth': 5,
 'max_features': 0.5,
 'min_samples_leaf': 5,
 'min_samples_split': 5,
 'n_estimators': 100}

In [211]:
def cal_k_result(model_source):
    kf=KFold(n_splits=5)
    train_loss=[]
    pred_result=[]
    for train_idx,test_idx in  kf.split(train_data):
        model=clone(model_source)
        X_kf_train=X_train[train_idx]
        y_kf_train=y_train[train_idx]
        X_kf_test=X_train[test_idx]
        y_kf_test=y_train[test_idx]
        model.fit(X_kf_train,ravel(y_kf_train))

        y_kf_predict=model.predict_proba(X_kf_test)
        weight=log_loss(y_kf_test,y_kf_predict)
        train_loss.append(1/weight+0.5)
        pred_result.append(model.predict_proba(X_test))
    return train_loss,pred_result

In [210]:
rf=RandomForestClassifier(
 max_depth= 5,
 max_features=0.5,
 min_samples_leaf=5,
 min_samples_split=5,
 n_estimators=100
)

In [212]:
kf_result=cal_k_result(rf)
total_weight=sum(kf_result[0])
final_result=0
for weight,result in zip(kf_result[0],kf_result[1]):
    final_result+=weight*result/total_weight

result=np.argmax(final_result,axis=1)
result_tocsv(result)

In [227]:
rf.fit(X_train,y_train)

D:\tech_softwar\miniconda\envs\for_autogluon\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(max_depth=5, max_features=0.5, min_samples_leaf=5,
                       min_samples_split=5)

In [230]:
rf.predict(X_train)!=

array([[False, False,  True, ...,  True,  True, False],
       [False, False,  True, ...,  True,  True, False],
       [ True,  True, False, ..., False, False,  True],
       ...,
       [ True,  True, False, ..., False, False,  True],
       [ True,  True, False, ..., False, False,  True],
       [False, False,  True, ...,  True,  True, False]])

In [239]:
wrong_list=rf.predict(X_train)!=ravel(y_train)

In [240]:
wrong_idx=[]
for idx,value in enumerate(wrong_list):
    if value:
        wrong_idx.append(idx)

In [243]:
X_train[wrong_idx]

array([[ 8.,  1.,  4., ...,  0.,  1.,  0.],
       [ 6.,  1.,  2., ...,  0.,  0.,  0.],
       [ 4.,  0.,  7., ...,  0.,  0.,  0.],
       ...,
       [11.,  1.,  0., ...,  0.,  0.,  0.],
       [ 2.,  0.,  7., ...,  0.,  0.,  0.],
       [ 3.,  0.,  5., ...,  0.,  0.,  0.]])

In [245]:
train_set.iloc[wrong_idx,:]

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
63,63,8.0,Yes,4.0,2.0,Yes,NaN,0.0,Extrovert
102,102,6.0,Yes,2.0,0.0,Yes,2.0,0.0,Extrovert
124,124,4.0,No,7.0,6.0,No,11.0,3.0,Introvert
139,139,11.0,NaN,2.0,2.0,Yes,2.0,2.0,Extrovert
149,149,0.0,No,4.0,6.0,No,NaN,3.0,Introvert
...,...,...,...,...,...,...,...,...,...
18337,18337,5.0,Yes,3.0,NaN,NaN,4.0,2.0,Extrovert
18377,18377,5.0,Yes,2.0,0.0,Yes,7.0,1.0,Extrovert
18437,18437,11.0,Yes,0.0,2.0,Yes,0.0,3.0,Extrovert
18443,18443,NaN,No,7.0,4.0,No,9.0,4.0,Introvert


In [246]:
train_set

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0.0,No,6.0,4.0,No,15.0,5.0,Extrovert
1,1,1.0,No,7.0,3.0,No,10.0,8.0,Extrovert
2,2,6.0,Yes,1.0,0.0,NaN,3.0,0.0,Introvert
3,3,3.0,No,7.0,3.0,No,11.0,5.0,Extrovert
4,4,1.0,No,4.0,4.0,No,13.0,NaN,Extrovert
...,...,...,...,...,...,...,...,...,...
18519,18519,3.0,No,7.0,3.0,No,9.0,7.0,Extrovert
18520,18520,1.0,NaN,6.0,7.0,No,6.0,5.0,Extrovert
18521,18521,7.0,Yes,1.0,1.0,Yes,1.0,NaN,Introvert
18522,18522,NaN,Yes,1.0,0.0,Yes,5.0,2.0,Introvert


# 带有源数据的feature

In [ ]:
pd.